In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from datetime import datetime
from tkinter import *
from tkinter import font
from tkinter import filedialog
import tkinter as tk
from tkinter import messagebox
import tkinter.simpledialog as sd
from ttkthemes import ThemedStyle
from PIL import Image, ImageTk

# Step 1: Dataset Preparation
dataset_path = 'C:\\Users\\SABIN\\Desktop\\onlycnn\\images'

# Step 2: Data Preprocessing
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
])

dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Split dataset into training and validation sets
val_size = int(0.2 * len(dataset))
train_size = len(dataset) - val_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Step 3: Model Definition
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(32 * 16 * 16, 128)  # Adjust the input size based on your image dimensions
        self.relu3 = nn.ReLU()
        
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        
        x = x.view(-1, 32 * 16 * 16)  # Adjust the size based on your image dimensions
        x = self.relu3(self.fc1(x))
        
        x = self.fc2(x)
        return x

# Step 4: Model Initialization
num_classes = len(dataset.classes)
cnn_model = SimpleCNN(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

# Step 5: Training Loop
num_epochs = 50
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    cnn_model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = cnn_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation loop
    cnn_model.eval()
    val_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for val_inputs, val_labels in DataLoader(val_dataset, batch_size=batch_size, shuffle=False):
            val_outputs = cnn_model(val_inputs)
            val_loss += criterion(val_outputs, val_labels).item()

            _, predicted = torch.max(val_outputs, 1)
            correct_predictions += (predicted == val_labels).sum().item()
            total_samples += val_labels.size(0)

    avg_val_loss = val_loss / len(val_dataset)
    accuracy = correct_predictions / total_samples

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}, Validation Loss: {avg_val_loss:.4f}, Accuracy: {accuracy * 100:.2f}%')

# Step 6: Save the Model
model_save_path = f'cnn_model_{datetime.now().strftime("%Y%m%d%H%M%S")}.pth'


try:
    torch.save(cnn_model.state_dict(), model_save_path)
    print(f"Model saved successfully at: {model_save_path}")
except Exception as e:
    print(f"Error saving the model: {e}")


Epoch [1/50], Loss: 0.8167, Validation Loss: 0.0323, Accuracy: 73.91%
Epoch [2/50], Loss: 0.4090, Validation Loss: 0.0152, Accuracy: 100.00%
Epoch [3/50], Loss: 0.1499, Validation Loss: 0.0059, Accuracy: 100.00%
Epoch [4/50], Loss: 0.0582, Validation Loss: 0.0027, Accuracy: 100.00%
Epoch [5/50], Loss: 0.0055, Validation Loss: 0.0008, Accuracy: 100.00%
Epoch [6/50], Loss: 0.0239, Validation Loss: 0.0004, Accuracy: 100.00%
Epoch [7/50], Loss: 0.0037, Validation Loss: 0.0016, Accuracy: 100.00%
Epoch [8/50], Loss: 0.0048, Validation Loss: 0.0000, Accuracy: 100.00%
Epoch [9/50], Loss: 0.0010, Validation Loss: 0.0006, Accuracy: 100.00%
Epoch [10/50], Loss: 0.0002, Validation Loss: 0.0001, Accuracy: 100.00%
Epoch [11/50], Loss: 0.0965, Validation Loss: 0.0000, Accuracy: 100.00%
Epoch [12/50], Loss: 0.0011, Validation Loss: 0.0024, Accuracy: 95.65%
Epoch [13/50], Loss: 0.3507, Validation Loss: 0.0001, Accuracy: 100.00%
Epoch [14/50], Loss: 0.0017, Validation Loss: 0.0108, Accuracy: 91.30%
Epoc

In [2]:
import torch
import torch.nn.functional as F
from PIL import Image
import cv2
import os
import time
import csv
from datetime import datetime
from torchvision import transforms
from torch.utils.data import DataLoader
from facenet_pytorch import InceptionResnetV1

# Step 1: Load the trained CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(32 * 16 * 16, 128)  # Adjust the input size based on your image dimensions
        self.relu3 = nn.ReLU()
        
        self.fc2 = nn.Linear(128, num_classes)
    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
    
    # Print the shape of x to understand its dimensions
        print(x.shape)
    
    # Dynamically adjust the size based on the shape of x
        x = x.view(x.size(0), -1)
    
    # Adjust the size of fc1 based on the number of features in x
        self.fc1 = nn.Linear(x.size(1), 128)
    
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        return x


# Assuming your CNN model is named 'cnn_model'
cnn_model = SimpleCNN(num_classes=num_classes)

# Load the trained weights
model_weights_path = 'C:/Users/SABIN/Desktop/onlycnn/cnn_model_20240302224000.pth'
cnn_model.load_state_dict(torch.load(model_weights_path))
cnn_model.eval()
#assuming
your_input_width = 100
your_input_height = 100
# Function to open the file and perform face recognition


def open_file():
    dataset = datasets.ImageFolder('C:\\Users\\SABIN\\Desktop\\onlycnn\\images')  # Provide the correct path
    idx_to_class = {i: c for c, i in dataset.class_to_idx.items()}

    def collate_fn(x):
        return x[0]

    loader = DataLoader(dataset, collate_fn=collate_fn)

    name_list = []  
    embedding_list = []

    for img, idx in loader:
        img_tensor = transforms.ToTensor()(img)
        
        # Use your CNN model for face recognition
        with torch.no_grad():
            outputs = cnn_model(img_tensor.unsqueeze(0))
        
        emb = outputs.detach()
        embedding_list.append(emb)
        name_list.append(idx_to_class[idx])

    # Save data
    data = [embedding_list, name_list]
    torch.save(data, 'data.pt')  # Saving data.pt file

    # Create or open the CSV file for attendance
    with open('recog.csv', mode='w', newline='') as csvfile:
        fieldnames = ['Name', 'Timestamp']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Write the header
        writer.writeheader()

        cam = cv2.VideoCapture(0)
        written_names = []

        while True:
            ret, frame = cam.read()
            if not ret:
                print("Fail to grab frame, try again")
                break

            # Perform face detection using OpenCV
            face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

            for (x, y, w, h) in faces:
                # Extract the face from the frame
                face = frame[y:y+h, x:x+w]
                # Resize the face to match the input size of your CNN
                face = cv2.resize(face, (your_input_width, your_input_height))

                # Convert the face to a PyTorch tensor
                img_tensor = transforms.ToTensor()(Image.fromarray(face))

                # Use your CNN model for face recognition
                with torch.no_grad():
                    outputs = cnn_model(img_tensor.unsqueeze(0))

                emb = outputs.detach()
                dist_list = []  # list of matched distances, minimum distance is used to identify the person

                for idx, emb_db in enumerate(embedding_list):
                    dist = torch.dist(emb, emb_db).item()
                    dist_list.append(dist)

                min_dist = min(dist_list)  # get minimum dist value
                min_dist_idx = dist_list.index(min_dist)  # get minimum dist index
                name = name_list[min_dist_idx]  # get name corresponding to minimum dist

                if min_dist < 0.90 and name not in written_names:
                    # Draw bounding box and label on the face
                    frame = cv2.putText(frame, f"{name} {min_dist:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                                        (0, 255, 0), 1, cv2.LINE_AA)
                    frame = cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
                    written_names.append(name)

                    # Write attendance data to CSV
                    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    writer.writerow({'Name': name, 'Timestamp': timestamp})

            cv2.imshow("IMG", frame)

            k = cv2.waitKey(1)
            if k % 256 == 27:  # ESC
                print('Esc pressed, closing...')
                break

        cam.release()
        cv2.destroyAllWindows()


                 
def show():
    loading_frame = tk.Frame(root, bg="#f7f7f7")
    loading_frame.pack(side=tk.BOTTOM)

    # Create a label to display the animation
    loading_label = tk.Label(loading_frame, text="Loading...", font=("Helvetica", 18), bg="#f7f7f7")
    loading_label.pack(padx=20, pady=20)

    # Use the after method to delay showing the animation for 2 seconds
    root.after(2000, lambda: loading_frame.pack_forget())
    # Replace this line with the code to display your data
    # For example:
    # data_label = tk.Label(root, text="Here's your data!", font=("Helvetica", 18))
    # data_label.pack(padx=20, pady=20)
    os.startfile('C:\\Users\\SABIN\\Desktop\\onlycnn\\recog.csv')
    

def help():
    help_text = """
    Welcome to the Swastik College Attendance System!

    This program is designed to help you keep track of attendance for your classes.

    To get started, simply click the Start button and select the file containing your class roster.

    Once you've selected the file, the program will automatically detect the names of your students and begin tracking attendance.

    Use the Stop button to end attendance tracking and save the results to a file.

    You can also use the Show button to display a summary of attendance data for your class.

    If you have any questions or issues, please don't hesitate to contact the support team at support@swastik.edu.np.
    """
    messagebox.showinfo("Help", help_text)
    import os
def add():
      
    # Load the Haar Cascade face classifier
    face_cascade = cv2.CascadeClassifier('C:\\Users\\SABIN\\Downloads\\haarcascade_frontalface_default.xml')

    # Initialize the video capture object
    cap = cv2.VideoCapture(0)

    # Prompt user for the person's name
    person_name = sd.askstring("Add Person", "Enter the person's name:")
    
    # Check if person_name is None (user clicked Cancel or closed the input prompt)
    if person_name is not None:
        # Add the person to the listbox
        listbox.insert(tk.END, person_name)

        # Create a folder for the person's face images
        folder_path = os.path.join('C:\\Users\\SABIN\\Desktop\\onlycnn\\images', person_name)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        # Capture 50 face images
        face = None
        count = 0
        while count < 50:
            # Capture frame-by-frame
            ret, frame = cap.read()

            # Convert the frame to grayscale
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Detect faces in the grayscale frame
            faces = face_cascade.detectMultiScale(gray, 1.3, 5)

            # Iterate over each detected face and save it as an image
            for (x, y, w, h) in faces:
                # Extract the face from the frame
                face = frame[y:y+h, x:x+w]

                # Resize the face to 100x100 pixels
                face = cv2.resize(face, (100, 100))

                # Save the face as an image in the person's folder
                cv2.imwrite(os.path.join(folder_path, 'face_{}.jpg'.format(count)), face)

                # Increment the count of captured faces
                count += 1

            # Display the captured face
            cv2.imshow('Captured Face', face)

            # Press 'q' to exit
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # Release the video capture object
        cap.release()

        # Close all windows
        cv2.destroyAllWindows()

    
       
root = tk.Tk()

# Set the window title and icon
root.title("7TH Project")
root.geometry("800x500")

# Set the window background color
root.config(bg="#cfe4f7")



# Create a top frame with a label
top_frame = tk.Frame(root, height=100)
top_frame.pack(fill=tk.X)

# Define gradient colors
gradient_colors = ("#34495E", "#2C3E50")

# Create a canvas for the gradient background
gradient_canvas = tk.Canvas(top_frame, width=root.winfo_screenwidth(), height=100, bg="#2C3E50", highlightthickness=0)
gradient_canvas.place(x=0, y=0)

# Create gradient rectangle
gradient_canvas.create_rectangle(0, 0, root.winfo_screenwidth(), 100, fill="", outline="")

# Add gradient colors to rectangle
for i in range(len(gradient_colors)):
    gradient_canvas.create_line(0, i, root.winfo_screenwidth(), i, fill=gradient_colors[i], width=1)
    
# Create a label on the top frame
title_label = tk.Label(top_frame, text="Attendance System", font=("Helvetica", 30), bg="#2C3E50", fg="#ECF0F1")
title_label.place(relx=0.5, rely=0.5, anchor="center")



# Create a frame for the logo image
logo_frame = tk.Frame(root, bg="#cfe4f7")
logo_frame.pack()

# Load the image using PIL
image = Image.open("C:\\Users\\SABIN\\Desktop\\logo.jpg")

# Scale the image to double the size
width, height = image.size
image = image.resize((int(width*0.8), int(height*0.8)))

# Create a PhotoImage object from the image
photo = ImageTk.PhotoImage(image)

# Add the image to the label in the logo frame
logo_label=tk.Label(logo_frame, image=photo, bg="#cfe4f7")
logo_label.pack(pady=20)

text_label = tk.Label(logo_frame, text="""7th SEMESTER PROJECT ON\nAutomated Attendance System using CNN\nSwastik College""",
                      font=("Helvetica", 18), bg="#cfe4f7", fg="#283142", justify="center", padx=20)
text_label.pack()


#Create a frame for the buttons
button_frame = tk.Frame(root, bg="#cfe4f7")
button_frame.pack()

# Create three buttons in the button frame
start_button = tk.Button(button_frame, text="Start", width=10, height=4, bg="#00a896", fg="#f7f7f7", font=("Helvetica", 18), command=open_file)
start_button.pack(side=tk.LEFT, padx=20, pady=20)

show_button = tk.Button(button_frame, text="Show", width=10, height=4, bg="#283142", fg="#f7f7f7", font=("Helvetica", 18), command=show)
show_button.pack(side=tk.LEFT, padx=20, pady=20)

help_button = tk.Button(button_frame, text="Help", width=10, height=4, bg="#00a896", fg="#f7f7f7", font=("Helvetica", 18), command=help)
help_button.pack(side=tk.LEFT, padx=20, pady=20)





# Create the listbox
listbox = tk.Listbox(root)
listbox.pack()

add_button = tk.Button(button_frame, text="Add", width=10, height=4, bg="#00a896", fg="#f7f7f7", font=("Helvetica", 18), command=add)
add_button.pack(side=tk.LEFT, padx=20, pady=20)


# Make the window resizable
root.resizable(True, True)

# Set the minimum window size
root.minsize(800, 500)

root.mainloop()

RuntimeError: Error(s) in loading state_dict for SimpleCNN:
	size mismatch for fc2.weight: copying a param with shape torch.Size([4, 128]) from checkpoint, the shape in current model is torch.Size([3, 128]).
	size mismatch for fc2.bias: copying a param with shape torch.Size([4]) from checkpoint, the shape in current model is torch.Size([3]).